In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [2]:
#import torch as nn
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoModel, TrainingArguments,Trainer
from datasets import Dataset, DatasetDict

2024-08-05 21:55:06.265193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 21:55:06.265305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 21:55:06.395915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Explore the Data

In [3]:
df = pd.read_csv("/kaggle/input/emotions/text.csv", index_col=[0], nrows=50000) # chunksize=10000
df

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
49995,i will flush all the crap and the fat out so i...,1
49996,i can think of a few things that i feel like w...,1
49997,i would recommend it to anyone who feels dissa...,3
49998,i think that what i m feeling here goes a lot ...,1


In [4]:
df.describe()

,label
count,50000.000000
mean,1.546280
std,1.492951
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [5]:
df.describe(include="object")

,text
count,50000
unique,49664
top,i feel fine
freq,3


In [6]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 50000
})

In [7]:
ds = ds.rename_column("__index_level_0__", "id")
ds = ds.rename_column("label", "labels") # huggingface trainer expects label column to be named labels

# Finetune with Distilbert

In [8]:
model_nm = "microsoft/deberta-v3-small"
tokenizer = AutoTokenizer.from_pretrained(model_nm)
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=6)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [10]:
tokenized_ds

Dataset({
    features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50000
})

In [11]:
split_ds = tokenized_ds.train_test_split(0.25)
split_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 37500
    })
    test: Dataset({
        features: ['text', 'labels', 'id', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12500
    })
})

# Finetuning

In [12]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [13]:
bs = 64
epochs = 4
lr = 8e-5

In [14]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

trainer = Trainer(model, args, train_dataset=split_ds['train'], eval_dataset=split_ds['test'],
                  tokenizer=tokenizer)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.605100,0.125570
2,0.124900,0.111621
3,0.102500,0.105204
4,0.084900,0.106227


TrainOutput(global_step=2344, training_loss=0.20735134280989195, metrics={'train_runtime': 650.7879, 'train_samples_per_second': 230.49, 'train_steps_per_second': 3.602, 'total_flos': 2152863754652880.0, 'train_loss': 0.20735134280989195, 'epoch': 4.0})

# Test Model

In [16]:
input_text = "i guess that feeling is what im really getting all nostalgic about"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Convert the tokenized input to a Dataset
eval_ds = Dataset.from_dict({
    
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask']
})
eval_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1
})

In [17]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[-2.48242188,  0.95507812,  8.4609375 , -2.61132812, -2.62109375,
        -2.79882812]])

In [18]:
import torch
import torch.nn.functional as F

# Apply softmax to convert logits to probabilities (optional, for interpretation)
probs = F.softmax(torch.tensor(preds), dim=-1)

# Get the predicted class (index of the largest logit)
predicted_class = torch.argmax(probs, dim=1).item()

# Print the predicted class
print(f"Predicted class: {predicted_class}")

Predicted class: 2
